In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from skimage.color import rgb2gray
from skimage.transform import rescale
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import cv2

In [2]:
img = list()
gray = list()
for i in range(441):
    img.append(cv2.imread(f'{i}.jpg'))
    gray.append(cv2.cvtColor(img[i], cv2.COLOR_BGR2GRAY))


img = np.array(img)
gray = np.array(gray)

In [3]:
descriptor = list()
bad_pic = list()
orb = cv2.ORB_create(nfeatures=100)
for i in range(len(gray)):
    kp1, des1 = orb.detectAndCompute(gray[i], None)
    if(type(des1) != type(None) and des1.shape == (100,32)):
        descriptor.append(des1.flatten())
    else:
        bad_pic.append(i)
        
        
descriptor = np.array(descriptor)

In [4]:
X = descriptor

In [5]:
target = pd.read_csv("coordinates.csv")
Y = np.array(target.values.tolist())

In [6]:
mask = np.ones(len(Y), dtype=bool)
for index in bad_pic:
    mask[index] = 0

In [7]:
Y = Y[mask]

In [8]:
model = Sequential()
model.add(Dense(1600, activation='relu',input_shape=X[0].shape))
model.add(Dense(200, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(2))

In [9]:
kf = KFold(n_splits=5, shuffle= True)

mse = list()
train = list()

for train_index, test_index in kf.split(X):
    x_train, x_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]
    
    model = Sequential()
    model.add(Dense(1600, activation='relu',input_shape=X[0].shape))
    model.add(Dropout(0.01))
    model.add(Dense(200, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(2))
    model.compile(loss='mse', optimizer='adam', metrics=['mean_squared_error'])
    model.fit(x_train, y_train, epochs=10, batch_size=3)
    
    results = model.predict(x_test)
    train_mse = model.predict(x_train)
    mse.append(mean_squared_error(y_test,results))
    train.append(mean_squared_error(y_train,train_mse))

Epoch 1/10
115/115 [==============================] - 3s 23ms/step - loss: 41982.3281 - mean_squared_error: 41982.3242 0s - loss: 46045.3281 - mean_squared_error: 
Epoch 2/10
115/115 [==============================] - 3s 23ms/step - loss: 8303.5898 - mean_squared_error: 8303.5898
Epoch 3/10
115/115 [==============================] - 3s 22ms/step - loss: 4320.6338 - mean_squared_error: 4320.6338
Epoch 4/10
115/115 [==============================] - 3s 23ms/step - loss: 3282.3591 - mean_squared_error: 3282.3591
Epoch 5/10
115/115 [==============================] - 3s 22ms/step - loss: 1747.6722 - mean_squared_error: 1747.6722
Epoch 6/10
115/115 [==============================] - 3s 22ms/step - loss: 1582.5231 - mean_squared_error: 1582.5231
Epoch 7/10
115/115 [==============================] - 3s 23ms/step - loss: 1080.6426 - mean_squared_error: 1080.6426
Epoch 8/10
115/115 [==============================] - 3s 23ms/step - loss: 1229.2583 - mean_squared_error: 1229.2583
Epoch 9/10
115/11

In [10]:
sum(mse)/5

3194.7805523394222

In [11]:
sum(train)/5

275.4714799710616